In [1]:
#===============================================================================================
import sys
sys.path.append(r"C:\Users\kid\SynologyDrive\Lab2023\KIDs\QTLab2324\IR_SING_PHOT\Synth")
sys.path.append(r"C:\Users\kid\SynologyDrive\Lab2023\KIDs\QTLab2324\IR_SING_PHOT\DAQ")
sys.path.append(r"C:\Users\kid\SynologyDrive\Lab2023\KIDs\QTLab2324\IR_SING_PHOT\API\logs")
sys.path.append(r'C:\Users\kid\SynologyDrive\Lab2023\KIDs\QTLab2324\IR_SING_PHOT\Exceptions')
sys.path.append(r"C:\Users\kid\SynologyDrive\Lab2023\KIDs\QTLab2324\IR_SING_PHOT\HDF5")
#===============================================================================================

import Synthesizer
from DAQ import DAQ
import json
from Continuous_Acquisition import ACQUISITION_CONFIG
import logging
from logging.config import dictConfig
from logs.logging_config import LOGGING_CONFIG
from Exceptions import replace_non_serializable
import numpy as np
import niscope as ni
import time

In [2]:
#===============================================================================================
#Save acquisition configuration parameters for DAQ configuration
#===============================================================================================

filepath = r"C:\Users\kid\SynologyDrive\Lab2023\KIDs\QTLab2324\IR_SING_PHOT\Runs"

#===============================================================================================
#Import logger configuration
#===============================================================================================
dictConfig(LOGGING_CONFIG)
logger = logging.getLogger(__name__)
logger.info('START EXECUTION')

devicename = 'PXI1Slot4'
sGen_board = ''


try:
    cfg1 = json.dumps(replace_non_serializable(ACQUISITION_CONFIG))
    logger.info('Dumping acquisition configuration')
    with open(ACQUISITION_CONFIG['acq_conf']['path'] + 'config.json','w') as f:
        f.write(cfg1)
except Exception:
    logger.critical('Dumping acquisition configuration')
    raise SystemError("Could not dump acquisition configuration!")

#===============================================================================================
#Acquire DAQ configuration dictionaries
#===============================================================================================

res = 5.345215443088618
sgen_board = ''
amplitude = 0
f = 0

try:
    daq = DAQ()
    logger.info('DAQ class object correctly created')
except Exception:
    logger.critical('Could not crate DAQ class object')
    raise SyntaxError('Could not create DAQ class object')

try:
    s1 = Synthesizer.Synthesizer(1)
    s1.connettore()
    logger.info('Synth class object correctly created and connected!')
except Exception:
    logger.critical('Could not crate synth class object')
    raise SyntaxError('Could not create synth class object')


channels = {'I'			: 0, 
            'Q'			: 1,
            'trigger'	: 0}


try:
    stat = daq.get_status
    logger.info('DAQ status: '+str(stat))
except Exception:
    logger.critical('Could not get DAQ status!')
    raise SystemError('Could not get DAQ status')

try:
    daq.reset_with_def()
    daq.get_status
    logger.info('Resetting DAQ with defaults')
except Exception:
    logger.error('Could not reset DAQ with defaults')
    pass

try:
    daq.acq_conf = ACQUISITION_CONFIG['acq_conf']
    logger.info('Inserting acquisition configuration from ACQUISITION_CONFIG dictionary')
except Exception:
    logger.warning('Coulkd not insert acquisition configuration')
    raise SystemError('Coulkd not insert acquisition configuration')

try:
    daq.horizontal_conf = ACQUISITION_CONFIG['horizontal']
    logger.info('Inserting hconf dic')
except Exception:
    logger.warning('Could not insert hconf dic')
    raise SystemError('Could not insert hconf dic')

try:
    daq.vertical_conf = ACQUISITION_CONFIG['vertical']
    logger.info('Inserting vconf dic')
except Exception:
    logger.warning('Could not insert vconf dic')
    raise SystemError('Could not insert vconf dic')

try:
    daq.chan_conf = ACQUISITION_CONFIG['chan_conf']
    logger.info('Inserting chan char dic')
except Exception:
    logger.warning('Could not insert chan char dic')
    raise SystemError('Could not insert chan char dic')

try:
    daq.trigger_dic = ACQUISITION_CONFIG['trigger']
    logger.info('Inserting trigger dic')
except Exception:
    logger.warning('Could not insert trigger dic')
    raise SystemError('Could not insert trigger dic')

11:57:51.234 - __main__ - <module> - INFO - START EXECUTION
11:57:51.237 - __main__ - <module> - INFO - Dumping acquisition configuration
11:57:51.245 - __main__ - <module> - INFO - DAQ class object correctly created
Synth_uno!
Si è connessi al synth_uno!
11:57:58.130 - __main__ - <module> - INFO - Synth class object correctly created and connected!
11:57:58.131 - __main__ - <module> - INFO - DAQ status: <bound method DAQ.get_status of <DAQ.DAQ object at 0x0000000008728B50>>
11:57:58.131 - __main__ - <module> - ERROR - Could not reset DAQ with defaults
11:57:58.132 - __main__ - <module> - INFO - Inserting acquisition configuration from ACQUISITION_CONFIG dictionary
11:57:58.132 - __main__ - <module> - INFO - Inserting hconf dic
11:57:58.134 - __main__ - <module> - INFO - Inserting vconf dic
11:57:58.135 - __main__ - <module> - INFO - Inserting chan char dic
11:57:58.135 - __main__ - <module> - INFO - Inserting trigger dic


In [3]:
s1.set_frequency(res)
s1.outp_frequency_on()

In [4]:
#===============================================================================================
#GET DATA!
#===============================================================================================

runs = 1000 # runs number        

daq._session = ni.Session(devicename)
logger.info('Creating new session!!!')

with daq._session as session:
    logger.info('Configuring channels')
    daq.configure_channels()
    daq.config_trigger()
    data = {'CH0': [],
            'CH1': [],
            'CH2': [],
            'CH3': []}
    I, Q = [], []
    
    for run in range(runs):
        time.sleep(0.1)
        print(run)
        s1.set_frequency(res)
        #logger.info(f'Synth 1 is now outputting signal at {res} GHz')
        daq._session.initiate()
        #logger.info('Initiating fetching...')
        I.extend(session.channels[0].fetch())
        Q.extend(session.channels[1].fetch())

11:58:31.524 - __main__ - <module> - INFO - Creating new session!!!
11:58:31.525 - __main__ - <module> - INFO - Configuring channels
0
11:58:31.680 - __main__ - <module> - INFO - Synth 1 is now outputting signal at 5.345215443088618 GHz
11:58:31.763 - __main__ - <module> - INFO - Initiating fetching...
1
11:58:32.149 - __main__ - <module> - INFO - Synth 1 is now outputting signal at 5.345215443088618 GHz
11:58:32.151 - __main__ - <module> - INFO - Initiating fetching...
2
11:58:32.258 - __main__ - <module> - INFO - Synth 1 is now outputting signal at 5.345215443088618 GHz
11:58:32.260 - __main__ - <module> - INFO - Initiating fetching...
3
11:58:32.368 - __main__ - <module> - INFO - Synth 1 is now outputting signal at 5.345215443088618 GHz
11:58:32.370 - __main__ - <module> - INFO - Initiating fetching...
4
11:58:32.477 - __main__ - <module> - INFO - Synth 1 is now outputting signal at 5.345215443088618 GHz
11:58:32.479 - __main__ - <module> - INFO - Initiating fetching...
5
11:58:32.5

In [5]:
data = {
    'I': [],
    'Q': []
}
for i in range(len(I)):
    data['I'].extend(I[i].samples.tolist())
    data['Q'].extend(Q[i].samples.tolist())

In [6]:
I_mean = np.array(data['I']).mean()
Q_mean = np.array(data['Q']).mean()
I_std = np.array(data['I']).std()
Q_std = np.array(data['Q']).std()

In [7]:
print('I_mean = ', I_mean)
print('Q_mean = ', Q_mean)
print('I_std = ', I_std)
print('Q_std = ', Q_std)

I_mean =  0.0028236391607332737
Q_mean =  -0.011031331974068265
I_std =  0.0009981187688265032
Q_std =  0.0007694430048609094
